In [ ]:
%reload_ext autoreload
%autoreload 2

from model_packages import *
from model_config import *
from model_utils import *

In [ ]:
os.chdir('..')

# Model Development

In [ ]:
# load counter data
ne_counters= gpd.read_file(ne_countr_locn_file)
ndw_counters= gpd.read_file(ndw_countr_locn_file)
crt_counters= gpd.read_file(crt_countr_locn_file)

In [ ]:
# list of sites from each counter source
places_ne= list(ne_counters['counter'].str.replace(' ', '_'))
places_ne[19]='Sandwich_Peninsula_Bridge'
places_nd= list(ndw_counters['counter'].str.replace(' ', '_'))
places_cr= list(crt_counters['Counter'].str.replace(' ', '_'))

## Multivariate regression model

<a id='dest_7'></a>

**Variable selection**

In [ ]:
df_sites_ne_nd_cr=pd.read_pickle(data_folder+'complete_dataset.pkl')

# Data for model training

df_for_model_training=df_sites_ne_nd_cr[df_sites_ne_nd_cr['site'].isin(places_ne+places_nd)]


print('Total number of sites for model training {}'.format(df_for_model_training['site'].unique().shape[0]))



df_for_model_estimation=df_sites_ne_nd_cr[df_sites_ne_nd_cr['site'].isin(places_cr)]


print('Total number of sites for model estimation {}'.format(df_for_model_estimation['site'].unique().shape[0]))

**Some descriptive stat to choose reference category for land use and land habitat features**

In [ ]:
site_classf=df_for_model_training[['site',"land_type_labels","habitat_type_labels"]].drop_duplicates()

print(site_classf['habitat_type_labels'].value_counts())

print(site_classf['land_type_labels'].value_counts())

# Not all classes are present in the training data
#df_for_model_training.value_counts("land_type_labels")
#df_for_model_training.value_counts("habitat_type_labels")

sns.violinplot(data=df_for_model_training, y="people_counter_data",x="land_type_labels",hue='habitat_type_labels')
plt.ylabel('People counter')
plt.show()

sns.violinplot(data=df_for_model_training, y="total_trip_count", x="land_type_labels",\
            hue="habitat_type_labels")
plt.ylabel('Strava count')
plt.show()

In [ ]:
site_classf=df_for_model_estimation[['site',"land_type_labels","habitat_type_labels"]].drop_duplicates()

print(site_classf['habitat_type_labels'].value_counts())

print(site_classf['land_type_labels'].value_counts())

df_for_model_estimation.value_counts("land_type_labels")
df_for_model_estimation.value_counts("habitat_type_labels")

sns.violinplot(data=df_for_model_estimation, y="total_trip_count", x="land_type_labels",\
            hue="habitat_type_labels")
plt.ylabel('Strava count')
plt.show()

**To deal with class imbalance- we combine the lowest minority classes into one class**

In [ ]:
hbt_dict={'Grassland_woodland_bareground':'Grass_wood_bareground_coast',\
          'Grassland_woodland_coastal':'Grass_wood_bareground_coast'}

lnd_dict={'rural_mixed_settings':'urban_and_rural_setings',\
          'urban_settings':'urban_and_rural_setings'}



df_for_model_training['habitat_type_labels'].replace(hbt_dict,inplace=True)

df_for_model_training['land_type_labels'].replace(lnd_dict,inplace=True)



df_for_model_estimation['habitat_type_labels'].replace(hbt_dict,inplace=True)

df_for_model_estimation['land_type_labels'].replace(lnd_dict,inplace=True)

In [ ]:
site_classf=df_for_model_training[['site',"land_type_labels","habitat_type_labels"]].drop_duplicates()

print(site_classf['habitat_type_labels'].value_counts())

print(site_classf['land_type_labels'].value_counts())

# Not all classes are present in the training data

#df_for_model_training.value_counts("land_type_labels")
#df_for_model_training.value_counts("habitat_type_labels")

sns.violinplot(data=df_for_model_training, y="people_counter_data",x="land_type_labels",hue='habitat_type_labels')
plt.ylabel('People counter')
plt.show()

sns.violinplot(data=df_for_model_training, y="total_trip_count", x="land_type_labels",\
            hue="habitat_type_labels")
plt.ylabel('Strava count')
plt.show()

In [ ]:
site_classf=df_for_model_estimation[['site',"land_type_labels","habitat_type_labels"]].drop_duplicates()

print(site_classf['habitat_type_labels'].value_counts())

print(site_classf['land_type_labels'].value_counts())

# Not all classes are present in the training data

#df_for_model_training.value_counts("land_type_labels")
#df_for_model_training.value_counts("habitat_type_labels")


sns.violinplot(data=df_for_model_estimation, y="total_trip_count", x="land_type_labels",\
            hue="habitat_type_labels")
plt.ylabel('Strava count')
plt.show()

In [ ]:
# load demographic features
demo_data= pd.read_pickle(data_folder+'socio_economic_data_all_sites.pkl')
# load POI features
pois_data= pd.read_pickle(data_folder+'pois_data_all_sites.pkl')

In [ ]:
df_pois_demo_all_sites=pois_data.merge(demo_data,left_on=['site'],\
                                               right_on=['counter'],how='inner')


df_pois_demo_all_sites=df_pois_demo_all_sites._get_numeric_data()

In [ ]:
trgt_ftrs=['people_counter_data']


dynmc_ftrs=['total_trip_count']

wthr_ftrs=['tavg']

othr_ftrs=['Mean_dog_occupancy']
           
    
lnd_ftrs=['land_type_labels_'+x for x in list(df_for_model_training['land_type_labels'].unique())]

hbt_ftrs=['habitat_type_labels_'+x for x in list(df_for_model_training['habitat_type_labels'].unique())]
    
catg_ftrs=['Date', 'site']
           



natr_ftrs_non_corr=['accessible_green_space_area','PROW_Total_length_km','waterside_length_km']

corr_cens_pois=df_pois_demo_all_sites.columns.tolist()

#non_corr_cens_pois=sel_demo_pois_vrbls

In [ ]:
pd.concat([df_for_model_training,df_for_model_estimation]).reset_index(drop=True).\
to_pickle(data_folder+'complete_dataset.pkl')

In [ ]:
df_for_model_training=pd.get_dummies(df_for_model_training,columns=['land_type_labels'])

df_for_model_training=pd.get_dummies(df_for_model_training,columns=['habitat_type_labels'])

In [ ]:
df_for_model_estimation=pd.get_dummies(df_for_model_estimation,columns=['land_type_labels'])

df_for_model_estimation=pd.get_dummies(df_for_model_estimation,columns=['habitat_type_labels'])

In [ ]:
df_for_model_training.to_pickle(data_folder+'df_for_model_training.pkl')

df_for_model_estimation.to_pickle(data_folder+'df_for_model_estimation.pkl')

**Linear regression model with regularisation**

In [ ]:
df_train, df_val= train_test_split(df_for_model_training, test_size=0.3, random_state=0,\
                                   stratify=df_for_model_training[['site']])

In [ ]:
df_train.to_pickle(data_folder+'sites_training_data.pkl')

df_val.to_pickle(data_folder+'sites_validation_data.pkl')

In [ ]:
ftrs_to_keep=trgt_ftrs+dynmc_ftrs+wthr_ftrs+othr_ftrs+natr_ftrs_non_corr+\
lnd_ftrs+hbt_ftrs+catg_ftrs+corr_cens_pois

ftrs_to_keep=[x for x in ftrs_to_keep if x in list(df_train.columns)]
print(ftrs_to_keep)

In [ ]:
# Create a custom palette
cmap = sns.diverging_palette(h_neg= 250, h_pos= 15, s=75, l=40, n=9, center="light", as_cmap=True)


# Compute corr matrix
matrix = pois_data.corr(method="pearson").dropna(how='all',axis=1).dropna(how='all',axis=0)
# Create a mask

# matrix=matrix[abs(matrix)>0.7].fillna(0.)
# mask = np.triu(np.ones_like(matrix, dtype=bool))

plt= sns.clustermap(matrix, annot=True,figsize=(10,10),\
               dendrogram_ratio=0.15,cmap=cmap,square=True,fmt=".2f",annot_kws={"size": 10}, vmin=-1, vmax=1)


plt.savefig(f"./outputs/poi_heatmap.png", format= 'png', dpi=300, bbox_inches='tight')

In [ ]:
clstr_labls=['habitat_type_labels_Grassland_woodland_wetland','land_type_labels_rural_settings']



ftrs_to_keep=[x for x in ftrs_to_keep if x not in clstr_labls]

In [ ]:
# For each ftr, calculate VIF and save in dataframe
vif = pd.DataFrame()

df_train_num=df_train[[x for x in ftrs_to_keep if x not in [target]]].select_dtypes(include=np.number)
vif["VIF Factor"] = [variance_inflation_factor(df_train_num.values, i) for \
                     i in range(df_train_num.shape[1])]
vif["features"] = df_train_num.columns


print(vif)

print('+'*100)

ftrs_to_chck=[x for x in ftrs_to_keep if x not in [target,'total_trip_count','tavg']]
              
df_train_num_remv_multi_coll=calculate_vif_(df_train[ftrs_to_chck].select_dtypes(include=np.number),\
                                            thresh=10)[0]

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_train_num_remv_multi_coll.values, i) for \
                     i in range(df_train_num_remv_multi_coll.shape[1])]
vif["features"] = df_train_num_remv_multi_coll.columns


#Vif removed features
low_vif_ftrs_df_train_num=list(vif.features.values)

# possibly correlated features
#low_vif_ftrs_df_train_num=ftrs_to_chck
              
low_vif_ftrs_df_train_num=low_vif_ftrs_df_train_num+['total_trip_count','tavg']
print(low_vif_ftrs_df_train_num)

In [ ]:
# Create a custom palette
cmap = sns.diverging_palette(250, 15, s=75, l=40, n=9, center="light", as_cmap=True)


# Compute corr matrix
matrix = df_for_model_training[low_vif_ftrs_df_train_num+[target]].corr(method="pearson").dropna(how='all',axis=1).dropna(how='all',axis=0)
# Create a mask

matrix=matrix[abs(matrix)>0.5].fillna(0.)
mask = np.triu(np.ones_like(matrix, dtype=bool))

sns.clustermap(matrix.sort_values(by='people_counter_data',ascending=False), annot=True,figsize=(10,10),\
               dendrogram_ratio=0.1,cmap=cmap,square=True,fmt=".2f",annot_kws={"size": 6}, vmin=-1, vmax=1)
plt.show();

In [ ]:
target='people_counter_data'


ftrs_to_keep=low_vif_ftrs_df_train_num+[target]

ftrs_to_keep_df=pd.DataFrame(ftrs_to_keep).rename(columns={0:'features_for_model'})

ftrs_to_keep_df.to_pickle(data_folder+'ftrs_to_keep_df.pkl')


# remove feature strava 
#ftrs_to_keep=[x for x in ftrs_to_keep if x not in ['total_trip_count','tavg']]

In [ ]:
print(ftrs_to_keep)

In [ ]:
df=df_train[ftrs_to_keep].copy()#ne_counter_strava_mdl_amenity_wthr_pop


features = [x for x in df.columns if x not in ['Date','site',target]]

categorical_columns = []
numerical_columns = features

preprocessor = make_column_transformer(
    (OneHotEncoder(drop="if_binary"), categorical_columns),
    remainder="passthrough",
    verbose_feature_names_out=False,  
)

np.random.seed(8)
scaler = RobustScaler()#/ 

rgr=ElasticNet()
param_distributions_rgrn={'alpha' : np.linspace(0.05, 5, 100),'l1_ratio':np.linspace(0.01, 1, 50)}
            
    
# if scaling the data    
scaler.fit(df[features])  
        
df[features] = scaler.transform(df[features])


# save the scaler
dump(scaler, open(data_folder+'scaler.pkl', 'wb'))


cv=RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
        
rs = RandomizedSearchCV(estimator=rgr, n_iter=2000,param_distributions=param_distributions_rgrn,cv=cv,\
                        scoring='explained_variance')


print(df[features].values.shape)
print(len(features))
X_train = df[features].values.reshape(-1, len(features))
y_train = df[target].values

rs.fit(X_train, y_train)
            
     
print(rs.best_score_)
            
print(rs.best_params_)     



# Getting the best estimator

alpha_reg=rs.best_params_.get('alpha')
l1_reg=rs.best_params_.get('l1_ratio')
model_regularisation = make_pipeline(preprocessor,
    TransformedTargetRegressor(
        regressor=rs.best_estimator_
    ),
)

model_regularisation.fit(df[features], df[target])

# save the model
dump(model_regularisation, open(data_folder+'model_regularisation.pkl', 'wb'))

y_pred = model_regularisation.predict(X_train)

y_pred =[max(0,x) for x in y_pred]

mae = median_absolute_error(y_train, y_pred)
string_score = f"MAE on training set: {mae:.2f}"

fig, ax = plt.subplots(figsize=(5, 5))
plt.scatter(y_train, y_pred)
ax.plot(transform=ax.transAxes, ls="--", c="red")
plt.text(3, 20, string_score)
plt.title("ElasticNet, regularization with alpha {} and l_1 ratio {}".format(np.round(alpha_reg,2),\
                                                                             np.round(l1_reg,2)))
plt.ylabel("Model predictions")
plt.xlabel("Truths")

print('r2_score on train data {}'.format(r2_score(y_train,y_pred)))

In [ ]:
model_regularisation

rs

In [ ]:
# load the model

model_regularisation = pickle.load(open(data_folder+'model_regularisation.pkl', 'rb'))
# load the scaler
scaler = pickle.load(open(data_folder+'scaler.pkl', 'rb'))

df_valid=df_val[ftrs_to_keep].copy() 

df_valid[features] = scaler.transform(df_valid[features])

X_valid = df_valid[features].values.reshape(-1, len(features))

y_valid = df_valid[target].values

y_pred_valid = model_regularisation.predict(X_valid)

y_pred_valid =[max(0,x) for x in y_pred_valid]

mae = median_absolute_error(y_valid, y_pred_valid)
string_score = f"MAE on validation set: {mae:.2f}"

fig, ax = plt.subplots(figsize=(5, 5))
plt.scatter(y_valid, y_pred_valid)
ax.plot(transform=ax.transAxes, ls="--", c="red")
plt.text(3, 20, string_score)
plt.title("ElasticNet, regularization with alpha {} and l_1 ratio {}".format(np.round(alpha_reg,2),\
                                                                             np.round(l1_reg,2)))
plt.ylabel("Model predictions")
plt.xlabel("Truths")

print('r2_score on valid data {}'.format(r2_score(y_valid,y_pred_valid)))

**Stability of predictors**

https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html

We can check the coefficient variability through cross-validation: it is a form of data perturbation (related to resampling).

If coefficients vary significantly when changing the input dataset their robustness is not guaranteed, and they should probably be interpreted with caution.

In [ ]:
feature_names = model_regularisation[:-1].get_feature_names_out()

coefs = pd.DataFrame(
    model_regularisation[-1].regressor_.coef_,
    columns=["Coefficients"],
    index=feature_names,
)

coefs.plot(kind='barh')

In [ ]:
coefs.plot.barh(figsize=(9, 7))
plt.title("ElasticNet, regularization with alpha {} and l_1 ratio {}".format(np.round(alpha_reg,2),\
                                                                             np.round(l1_reg,2)))
plt.axvline(x=0, color=".5")
plt.xlabel("Raw coefficient values")
plt.subplots_adjust(left=0.3)

In [ ]:
X_train_preprocessed = pd.DataFrame(
    model_regularisation[:-1].transform(X_train), columns=feature_names
)

X_train_preprocessed.std(axis=0).plot.barh(figsize=(9, 7))
plt.title("Feature ranges")
plt.xlabel("Std. dev. of feature values")
plt.subplots_adjust(left=0.3)

In [ ]:
coefs = pd.DataFrame(
    model_regularisation[-1].regressor_.coef_ * X_train_preprocessed.std(axis=0),
    columns=["Coefficient importance"],
    index=feature_names,
)
coefs.plot(kind="barh", figsize=(9, 7))
plt.xlabel("Coefficient values corrected by the feature's std. dev.")
plt.title("ElasticNet, regularization with alpha {} and l_1 ratio {}".format(np.round(alpha_reg,2),\
                                                                             np.round(l1_reg,2)))
plt.axvline(x=0, color=".5")
plt.subplots_adjust(left=0.3)

In [ ]:
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=0)
cv_model = cross_validate(
    model_regularisation,
    df[features],
    df[target],
    cv=cv,
    return_estimator=True,
    n_jobs=2,
)

coefs = pd.DataFrame(
    [
        est[-1].regressor_.coef_ * est[:-1].transform(df.iloc[train_idx]).std(axis=0)
        for est, (train_idx, _) in zip(cv_model["estimator"], cv.split(df[features], df[target]))
    ],
    columns=feature_names,
)

In [ ]:
coefs=coefs.reindex(sorted(coefs.columns), axis=1)

In [ ]:
plt.figure(figsize=(10,5))
sns.stripplot(data=coefs, orient="h", color="k", alpha=0.5)
sns.boxplot(data=coefs, orient="h", color="cyan", saturation=0.5, whis=10)
plt.axvline(x=0, color=".5")
plt.xlabel("Coefficient importance")
plt.title("Coefficients importance and their variability")
plt.suptitle("ElasticNet, regularization with alpha {} and l_1 ratio {}".format(np.round(alpha_reg,2),\
                                                                             np.round(l1_reg,2)))
plt.subplots_adjust(left=0.3)

**Linear regression without regularisation**

In [ ]:
X,y=df_train[features].copy(), df_train[target].copy()

X_valid,y_valid=df_val[features].copy(), df_val[target].copy()

X_regr=X.copy()
y_regr=y.copy()

X_regr_valid=X_valid.copy()
y_regr_valid=y_valid.copy()


sclng_flg=True

if sclng_flg:
    scaler_lin = StandardScaler()#/ 
    scaler_lin.fit(X_regr)  # 
    X_regr = scaler_lin.transform(X_regr)
    
    X_regr_valid = scaler_lin.transform(X_regr_valid)


ols = linear_model.LinearRegression()
model = ols.fit(X_regr, y_regr)

print('Features                :  %s' % features)
print('Regression Coefficients : ', [round(item, 4) for item in model.coef_])
print('R-squared               :  %.4f' % model.score(X_regr, y_regr))
print('Y-intercept             :  %.4f' % model.intercept_)
print('')

print('+'*200)


y_pred=[max(0,x) for x in model.predict(X_regr)]


print('r2_score on train data {}'.format(r2_score(y_regr,y_pred)))

plt.scatter(y_regr,y_pred)

y_pred_valid=[max(0,x) for x in model.predict(X_regr_valid)]

plt.scatter(y_regr_valid,y_pred_valid)

print('r2_score on valid data {}'.format(r2_score(y_regr_valid,y_pred_valid)))

In [ ]:
#add constant to predictor variables
x = sm.add_constant(X_regr)

#fit linear regression model
model = sm.OLS(y_regr, x).fit()

#view model summary
print(model.summary())

In [ ]:
# Getting regression coefficients as table
lm = pg.linear_regression(pd.DataFrame(X_regr,columns=features), y_regr)

lm

In [ ]:
# Creating Latex table of model coefficients

# ft_table= lm.copy()
# ft_table.names=ft_table.names.str.replace("_", " ")

# print(ft_table.to_latex(index=False,
#                   formatters={"name": str.upper},
#                   float_format="{:.2f}".format,
#                   longtable= True
#                   ))

In [ ]:
# Linear regression Model performance using cross validation

#https://scikit-learn.org/stable/modules/cross_validation.html

print(metrics.SCORERS.keys())
rgr_pip = make_pipeline(scaler_lin,LinearRegression())


cv = ShuffleSplit(n_splits=10, test_size=0.25, random_state=0)

scores=cross_val_score(rgr_pip, X,y, cv=cv,scoring='explained_variance')

print(scores)

print(np.mean(scores))

scores=cross_val_score(rgr_pip, X,y, cv=cv,scoring='neg_mean_absolute_error')

print(scores)

**MODEL COMPARISONS**

*We compare performance of different models*

<a id='dest_addtnl'></a>

In [ ]:
dataset=pd.read_pickle(data_folder+'sites_training_data.pkl')

kept_ftrs_train=dataset[['Date','site',target]]

ftrs_to_keep=pd.read_pickle(data_folder+'ftrs_to_keep_df.pkl')['features_for_model'].values

dataset=dataset[ftrs_to_keep]



print(ftrs_to_keep)

In [ ]:
# Transform dataset (transform, bin and create dummy variables) and split the dataset. In addition,
# we are logging experiments and plots for those experiment to be viewed later with MLflow. 
reg_fcr = setup(data=dataset, target=target, session_id=786, transformation=False,\
                normalize=False, train_size=0.7,\
                numeric_features=[x for x in list(dataset.select_dtypes(include=np.number).columns)\
                                  if x not in [target]],\
                remove_outliers=False,\
                outliers_threshold=0.05, remove_multicollinearity=True,\
                multicollinearity_threshold=0.9,feature_selection=False,\
                ignore_features=['Date','site'],\
                polynomial_features=False,feature_selection_method='classic',\
                pca=False, log_experiment=True,\
                experiment_name='reg_experiments', log_plots=True)

In [ ]:
models()

In [ ]:
# We can specify which models to compare or let the best model be selected 
#top5 = compare_models(n_select=5, exclude=(['ransac', 'knn']), sort='RMSE', fold=5)

top5 =compare_models(n_select=5, include=(['lr','ridge','lasso','en','br','kr']),sort='R2', fold=5)

In [ ]:
pull().sort_values(by='R2', ascending=True)

In [ ]:
tuned_top5 = [tune_model(i, n_iter=120, optimize='R2', fold=5) for i in top5]

In [ ]:
blender_specific = blend_models(estimator_list=tuned_top5[0:], fold=5, optimize='R2', choose_better=True)

In [ ]:
# Below is a view of the model parameters. 
blender_specific

In [ ]:
# Stacking models is an ensemble method of using meta learning, where a meta model is created 
# using multiple base estimators to generate the final prediction.
# Let's try building a stacking model from our top 5 models and evaluate the results. 
stacker_specific = stack_models(estimator_list=tuned_top5[1:], meta_model=tuned_top5[0],\
                                fold=5,optimize='R2', choose_better=True)

In [ ]:
# Below is a view of the model parameters. 
stacker_specific

In [ ]:
# We can use Pycaret's built in plot_model() function to generate side-by-side plots: the Residuals chart, Prediction Error and Cross Validation (learning) charts. Let's compare the Blend and Stack model plots in a side-by-side comparison. 
#fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(2,2,1)
plot_model(blender_specific, plot='residuals', verbose=False, scale=1.1)
ax = fig.add_subplot(2,2,2)
plot_model(stacker_specific, plot='residuals', verbose=False, scale=1.1)
ax = fig.add_subplot(2,2,3)
plot_model(blender_specific, plot='error', verbose=False, scale=1.1)
ax = fig.add_subplot(2,2,4)
plot_model(stacker_specific, plot='error',  verbose=False, scale=1.1)
#ax = fig.add_subplot(3,2,5)
#plot_model(blender_specific, plot='learning',verbose=False, scale=1.1)
#ax = fig.add_subplot(3,2,6)
#plot_model(stacker_specific, plot='learning', verbose=False, scale=1.1)
#plt.savefig('plots_blender_vs_stacker.png', dpi=300, pad_inches=0.25)
plt.show()

In [ ]:
# We can execute the predict_model() function to use the model to generate the predicted values. 
pred_tunded_blender = predict_model(blender_specific)

In [ ]:
# We can execute the predict_model() function to use the model to generate the predicted values. 
pred_tunded_stacker = predict_model(stacker_specific)

In [ ]:
# The Blend model seems to perform better in both our train and test so let us finalize it.
#The finalize_model() function trains the model on the entire dataset. 
finalize_blender = finalize_model(blender_specific)#

In [ ]:
save_model(finalize_blender, data_folder+'pycaret_regression_model') 

In [ ]:
pycaret_model = load_model(data_folder+'pycaret_regression_model') 

In [ ]:
# The predict_model() can be executed with the final blend model over 
# the entire dataset and saved to a csv file. 
pred_on_train = predict_model(pycaret_model, data=dataset[[x for x in dataset.columns if x not in ['people_counter_data']]])

pred_on_train['prediction_label']=pred_on_train['prediction_label'].clip(0)

pred_on_train=pd.concat([pred_on_train.reset_index(drop=True),kept_ftrs_train.reset_index(drop=True)],axis=1)

In [ ]:
plt.scatter(pred_on_train['people_counter_data'],pred_on_train['prediction_label'])

print('r2_score is {}'.format(r2_score(pred_on_train[target],pred_on_train['prediction_label'])))

In [ ]:
dataset_valid=pd.read_pickle(data_folder+'sites_validation_data.pkl').reset_index(drop=True)

kept_ftrs_valid=dataset_valid[['Date','site',target]]

dataset_valid=dataset_valid[list(ftrs_to_keep)]#+['Date','site']]

y_valid=dataset_valid[target]

dataset_valid=dataset_valid[[x for x in dataset_valid.columns if x not in [target]]]

In [ ]:
# The predict_model() can be executed with the final blend model over 
# the entire dataset and saved to a csv file. 
pred_on_val = predict_model(pycaret_model, data=dataset_valid)

pred_on_val['prediction_label']=pred_on_val['prediction_label'].clip(0)

pred_on_val=pd.concat([pred_on_val.reset_index(drop=True),kept_ftrs_valid.reset_index(drop=True)],axis=1)

In [ ]:
plt.scatter(pred_on_val[target],pred_on_val['prediction_label'])
print('r2_score is {}'.format(r2_score(pred_on_val[target],pred_on_val['prediction_label'])))

In [ ]:
pred_on_train.to_pickle(data_folder+'pred_on_train_data.pkl')

pred_on_val.to_pickle(data_folder+'pred_on_val_data.pkl')

In [ ]:
predtcns_sites=pred_on_val.set_index('site')

predtcns_sites['Date']=pd.to_datetime(predtcns_sites['Date'])

predtcns_sites=predtcns_sites.sort_values(by='Date')


#sns.scatterplot(data=predtcns_sites, x="people_counter_data", y="prediction_label", hue="site")

In [ ]:
fig = px.scatter(predtcns_sites, x="people_counter_data", y="prediction_label", color=predtcns_sites.index)
fig.show()

In [ ]:
# data wrangling for visualisation 

predtcns_sites=predtcns_sites.reset_index()


predtcns_sites=pd.melt(predtcns_sites,id_vars=['site','Date'],var_name='visitors', value_name='value')

predtcns_sites

In [ ]:
# Ground truth for all sites and dates
predtcns_sites_a=predtcns_sites[predtcns_sites['visitors']=='people_counter_data']
predtcns_sites_a['site']=predtcns_sites_a['site']+'-'+predtcns_sites_a['visitors']

del predtcns_sites_a['visitors']

predtcns_sites_a=predtcns_sites_a.sort_values(by=['Date','site']).reset_index(drop=True)

print(predtcns_sites_a)

# Predictions for all sites and dates
predtcns_sites_b=predtcns_sites[predtcns_sites['visitors']=='prediction_label']
predtcns_sites_b['site']=predtcns_sites_b['site']+'-'+predtcns_sites_b['visitors']

del predtcns_sites_b['visitors']

predtcns_sites_b=predtcns_sites_b.sort_values(by=['Date','site']).reset_index(drop=True)

print(predtcns_sites_b)

# Strava count for all sites and dates
predtcns_sites_c=predtcns_sites[predtcns_sites['visitors']=='total_trip_count']
predtcns_sites_c['site']=predtcns_sites_c['site']+'-'+predtcns_sites_c['visitors']

del predtcns_sites_c['visitors']

predtcns_sites_c=predtcns_sites_c.sort_values(by=['Date','site']).reset_index(drop=True)

print(predtcns_sites_c)

In [ ]:
predtcns_sites_a.groupby('Date')['value'].sum().plot(style='-o',label='Actual count')

predtcns_sites_b.groupby('Date')['value'].sum().plot(style='--',c='r',label='Predicted count')

#(predtcns_sites_c.groupby('Date')['value'].sum()*10).plot(style='--',c='g',label='Strava count (scaled)')



plt.ylabel('Visits')
plt.legend()

In [ ]:
fig1 = px.line(predtcns_sites_a, x="Date", color="site",
             y='value',
             title="True count",markers=True
            )
fig1.update_xaxes(dtick="M1",tickformat="%b\n%Y")
#fig.show()
fig2 = px.line(predtcns_sites_b, x="Date", color="site",
             y='value',
             title="Predicted count",markers=True,
            )
fig2.update_xaxes(dtick="M1",tickformat="%b\n%Y")
#fig.show()
fig2.update_traces(patch={"line": {"dash": 'dot'}}) 

fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.show()

In [ ]:
# We can use the Pycaret's built-in plot_model() function to generate Residuals and
# Error plots for the finalized blend model. 
fig = plt.figure(figsize=(5,10))
ax = fig.add_subplot(2,1,1)
plot_model(pycaret_model, plot='residuals',  verbose=False, scale=1.1)
ax = fig.add_subplot(2,1,2)
plot_model(pycaret_model, plot='error',  verbose=False, scale=1.0)
plt.show()

In [ ]:
#pd.DataFrame({'Feature': get_config('X_train').columns, 'Value' : finalize_blender.coef_}).\
#sort_values(by='Value', ascending=False).set_index('Feature').plot(kind='barh')

#pd.DataFrame({'Feature': get_config('X_train').columns, 'Value' : stacker_specific.coef_}).\
#sort_values(by='Value', ascending=False).set_index('Feature').plot(kind='barh')

**Clustering on sites to identify similar sites (based on training data and only on static attributes)**

The idea is to identify nearest beighbour (similar site) for each site in the unseen sites (river and canal trust data for example)

In [ ]:
# read in all the static features in the data 
all_sites_model=pd.read_pickle(data_folder+'complete_dataset.pkl')

counter_ne=pd.read_pickle(data_folder+'counter_ne.pkl')

counter_nd=pd.read_pickle(data_folder+'counter_nd.pkl')

counter_cr=pd.read_pickle(data_folder+'counter_cr.pkl')


all_sites_model=pd.get_dummies(all_sites_model,columns=['habitat_type_labels'])

all_sites_model=pd.get_dummies(all_sites_model,columns=['land_type_labels'])

ftrs_to_keep=pd.read_pickle(data_folder+'ftrs_to_keep_df.pkl')['features_for_model'].values


ftrs_to_be_used=[x for x in ftrs_to_keep if x not in ['tavg','total_trip_count','people_counter_data']]+['site']

print(ftrs_to_be_used)

all_sites_model=all_sites_model[ftrs_to_be_used].drop_duplicates().reset_index(drop=True)

print(all_sites_model.shape)

all_sites_num_colmns=list(all_sites_model._get_numeric_data().columns)


#scale the data

scaler_data=MinMaxScaler()

all_sites_model[all_sites_num_colmns]=scaler_data.fit_transform(all_sites_model[all_sites_num_colmns])

In [ ]:
training_sites=list(pd.read_pickle(data_folder+'df_for_model_training.pkl')['site'].unique())

estm_sites=list(pd.read_pickle(data_folder+'df_for_model_estimation.pkl')['site'].unique())

In [ ]:
sites_train=all_sites_model[all_sites_model['site'].isin(training_sites)].reset_index(drop=True)

ftrs_train=sites_train[[x for x in sites_train.columns if x not in ['site']]]


sites_estimation=all_sites_model[all_sites_model['site'].isin(estm_sites)].reset_index(drop=True)


ftrs_estm=sites_estimation[[x for x in sites_estimation.columns if x not in ['site']]]

In [ ]:
# Create a K-D tree on the data points in the train set 
# and find the nearest neighbour for each site in unseen data
A = ftrs_train.values
tree = spatial.KDTree(A)

str_neghbrs=[]

negbrs_dist=[]

negbrs_valus=[]
for indx in range(ftrs_estm.shape[0]):
    
    # index of the nearest neighbour for each data point in the unseen-data
    str_neghbrs.append(tree.query(ftrs_estm.iloc[indx,:].values)[1])
    
    # distance of the nearest neighbour for each data point in the unseen-data
    negbrs_dist.append(tree.query(ftrs_estm.iloc[indx,:].values)[0])
    
    # nearest neighbour for each data point in the unseen-data
    negbrs_valus.append(A[tree.query(ftrs_estm.iloc[indx,:].values)[1]])
    

# Get the neighbour from the training sites for each site in the unseen data (data for estimation)
sites_estimation['neighbouring_site']=sites_train.iloc[str_neghbrs,:]['site'].values

sites_estimation['neighbour_distance']=negbrs_dist

site_loc_buffer=gpd.read_file(data_folder+accesb_area_file)



site_loc_buffer.geometry=site_loc_buffer.centroid

site_loc_buffer=site_loc_buffer.to_crs(crs_deg)

sites_estimation=sites_estimation[['site','neighbouring_site','neighbour_distance']].\
rename(columns={'site':'counter'}).merge(site_loc_buffer,on=['counter'],how='inner')

sites_estimation.rename(columns={'counter':'site'},inplace=True)

del sites_estimation['geom_type']
del sites_estimation['area']

sites_estimation

In [ ]:
# pick sites in the unseen data which are most similar to the 
# sites in the training data 

sites_estimation=sites_estimation.sort_values(by='neighbour_distance',ascending=True).reset_index(drop=True)

sites_estimation['neighbour_distance'].hist(bins=10)
plt.show()

sites_estimation=sites_estimation[sites_estimation['neighbour_distance'].\
                                  le(sites_estimation['neighbour_distance'].quantile(0.25))]

sites_estimation['neighbour_distance'].hist(bins=10)
plt.show()

In [ ]:
print('Number of sites in the data to be estimated {}'.format(sites_estimation['site'].unique().shape[0]))

print('Number of sites in the training data most similar {}'.format(sites_estimation['neighbouring_site'].unique().shape[0]))

In [ ]:
# Visualisation of all the counter locations
ax=counter_ne.plot(color='blue',label='Natural England sites :{}'.\
                             format(counter_ne.shape[0]))
contextily.add_basemap(ax,crs= crs_deg,source=contextily.providers.OpenStreetMap.Mapnik)

counter_nd.plot(ax=ax,color='red',label='North downs way sites :{}'.\
                                   format(counter_nd.shape[0]))
contextily.add_basemap(ax,crs= crs_deg,source=contextily.providers.OpenStreetMap.Mapnik)

counter_cr.plot(ax=ax,color='green',label='Canal river trust sites :{}'.\
                                  format(counter_cr.shape[0]))
contextily.add_basemap(ax,crs= crs_deg,source=contextily.providers.OpenStreetMap.Mapnik)



plt.legend()

In [ ]:
#visualisation of sites most similar to the spatial sites in the unseen data

sites_estimation.geometry=gpd.GeoDataFrame(sites_estimation).centroid.to_crs(crs_deg)
sites_estimation=gpd.GeoDataFrame(sites_estimation)
labels=sites_estimation['neighbouring_site'].values

# Group unseen sites by the label of the neighbouring site
hull=sites_estimation[["geometry"]].to_crs(crs_deg).dissolve(by=labels).convex_hull

trng_sites=pd.concat([counter_ne,counter_nd])

trng_sites=trng_sites[trng_sites['counter'].isin(labels)].reset_index(drop=True)

In [ ]:
ax=trng_sites.plot(color='green',label='Number of sites as potential labels for unseen data :{}'.\
                   format(trng_sites.shape[0]))
contextily.add_basemap(ax,crs= crs_deg,source=contextily.providers.OpenStreetMap.Mapnik)
plt.legend()

In [ ]:
sites_estimation.groupby('neighbouring_site')['site'].count().sort_values(ascending=False).plot(kind='barh')
plt.xlabel('Number of similar sites in the unseen data')

In [ ]:
# Focus on top-n sites (in the training data) with the 
# highest count of  similar sites found in the unseen data

most_similar_sites=list(sites_estimation.groupby('neighbouring_site')['site'].count().\
sort_values(ascending=False)[:10].index.values)

sites_estimation=sites_estimation[sites_estimation['neighbouring_site'].isin(most_similar_sites)]

labels=sites_estimation['neighbouring_site'].values

# Group unseen sites by the label of the neighbouring site
hull=sites_estimation[["geometry"]].to_crs(crs_deg).dissolve(by=labels).convex_hull

print(sites_estimation)
trng_sites=trng_sites[trng_sites['counter'].isin(most_similar_sites)]

In [ ]:
trng_sites.to_pickle(data_folder+'crt_and_similar_sites.pkl')

In [ ]:
from matplotlib import cm

n_colors = len(trng_sites['counter'].unique())
colours = cm.rainbow(np.linspace(0, 1, n_colors))

# color code each site (in the training set) and same color 
# for similar site found in the unseen data

color_dict = dict(zip(trng_sites['counter'].unique(),colours))

trng_sites['Color'] = trng_sites['counter'].map(color_dict)

sites_estimation['Color'] =sites_estimation['neighbouring_site'].map(color_dict)

In [ ]:
#Visualise sites and clusters they belong to

# Set up figure and axis
f, ax = plt.subplots(1, figsize=(5, 5))
# Plot individual locations
trng_sites.to_crs(crs_deg).plot(
    # Colour by cluster label
    column='counter',
    # Consider label as categorical
    categorical=False,
    # Add 50% of transparency
    alpha=0.95,
    # Include legend
    legend=True,
    # Draw on axis `ax`
    ax=ax,
    # Use circle as marker
    marker="o",
    # Position legend outside the map
    legend_kwds={"bbox_to_anchor": (1, 1)},
)

sites_estimation.to_crs(crs_deg).plot(
    # Colour by cluster label
    column=labels,
    # Consider label as categorical
    categorical=True,
    # Add 50% of transparency
    alpha=0.85,
    # Include legend
    legend=False,
    # Draw on axis `ax`
    ax=ax,
    # Use circle as marker
    marker="+",
    # Position legend outside the map
    legend_kwds={"bbox_to_anchor": (1, 1)},
)


# Plot convex hull polygons for each cluster label
# except that for -1 (observations classified as noise)
#hull.boundary.plot(color="k", ax=ax)
# Add basemap
contextily.add_basemap(
    ax,
    crs=sites_estimation.to_crs(crs_deg).crs.to_string(),
    source=contextily.providers.CartoDB.VoyagerNoLabels,
)
plt.title(label='Number of most similar sites in the un-seen data {} marked with +'.\
          format(sites_estimation['site'].unique().shape[0]))
# Remove axes
ax.set_axis_off();

In [ ]:
# Set up figure and axis
f, ax = plt.subplots(1, figsize=(10, 5))
# Plot individual locations

trng_sites.to_crs(crs_deg).plot(
    # Colour by cluster label
    column='counter',
    #color=trng_sites['Color'],
    cmap=colors.ListedColormap(list(color_dict.values())),
    # Consider label as categorical
    #categorical=False,
    # Add 50% of transparency
    alpha=0.95,
    # Include legend
    legend=True,
    # Draw on axis `ax`
    ax=ax,
    # Use circle as marker
    marker="o",
    # Position legend outside the map
    legend_kwds={"bbox_to_anchor": (1, 1)},
)

sites_estimation.to_crs(crs_deg).plot(
    # Colour by cluster label
    column='site',
    #color=trng_sites['Color'],
    cmap=colors.ListedColormap(list(sites_estimation['Color'].values)),
    # Consider label as categorical
    #categorical=False,
    # Add 50% of transparency
    alpha=0.95,
    # Include legend
    legend=False,
    # Draw on axis `ax`
    ax=ax,
    # Use circle as marker
    marker="+",
    # Position legend outside the map
    legend_kwds={"bbox_to_anchor": (1, 1)},
)



# Plot convex hull polygons for each cluster label
# except that for -1 (observations classified as noise)
#hull.boundary.plot(color="k", ax=ax)
# Add basemap
contextily.add_basemap(
    ax,
    crs=sites_estimation.to_crs(crs_deg).crs.to_string(),
    source=contextily.providers.CartoDB.VoyagerNoLabels,
)
# Remove axes
plt.title(label='Number of most similar sites in the un-seen data {} marked with +'.\
          format(sites_estimation['site'].unique().shape[0]))
ax.set_axis_off();

**Model inferences**

In [ ]:
data_for_estimation=pd.read_pickle(data_folder+'df_for_model_estimation.pkl')


data_for_estimation=data_for_estimation[list(ftrs_to_keep)+['Date','site']].reset_index(drop=True)


data_for_estimation=data_for_estimation[data_for_estimation['site'].isin(list(sites_estimation['site'].\
                                                                              unique()))].reset_index(drop=True)


identf_cols_to_be_added=data_for_estimation[['Date','site']]


ftrs_model=[x for x in data_for_estimation.columns if x not in ['Date','site',target]]

del data_for_estimation[target]


data_used_for_training=pd.read_pickle(data_folder+'df_for_model_training.pkl')


data_used_for_training=data_used_for_training[list(ftrs_to_keep)+['Date','site']].reset_index(drop=True)




data_used_for_training=data_used_for_training[data_used_for_training['site'].isin(most_similar_sites)].\
reset_index(drop=True)

identf_cols_to_be_added_train=data_used_for_training[['Date','site',target]]

In [ ]:
# Use trained model to make predictions:

pycaret_flag=True
# If using py-caret model

if pycaret_flag:
    
    pycaret_model = load_model(data_folder+'pycaret_regression_model') 
    
    
    
    estimation_trained_model = predict_model(pycaret_model, data=data_for_estimation[ftrs_model])
    
    estimation_trained_model['prediction_label']=estimation_trained_model['prediction_label'].clip(0)
    
    prediction_trained_model = predict_model(pycaret_model, data=data_used_for_training[ftrs_model])
    
    prediction_trained_model['prediction_label']=prediction_trained_model['prediction_label'].clip(0)


# If using sk-learn model   
    
sklrn_flag=False

if sklrn_flag:
    
    model_regularisation = pickle.load(open(data_folder+'model_regularisation.pkl', 'rb'))
    # load the scaler
    
    scaler = pickle.load(open(data_folder+'scaler.pkl', 'rb'))
    
    # using  trained model sklearn to infer visitors at 'new but similar' locations
    estimation_trained_model=data_for_estimation[ftrs_model].copy()

    df_estmn=data_for_estimation[ftrs_model].copy() 

    features_estmn = [x for x in df_estmn.columns if x not in ['Date','site',target]]

    df_estmn[features_estmn] = scaler.transform(df_estmn[features_estmn])

    X_estm = df_estmn[features_estmn].values.reshape(-1, len(features_estmn))

    y_estm= model_regularisation.predict(X_estm)

    y_estm =[max(0,x) for x in y_estm]


    estimation_trained_model['prediction_label']=y_estm
    
    
    
    # using  trained model sklearn
    
    prediction_trained_model=data_used_for_training[ftrs_model].copy()

    df_trng=data_used_for_training[ftrs_model].copy() 
    
    

    features_trng = [x for x in df_trng.columns if x not in ['Date','site',target]]

    df_trng[features_trng] = scaler.transform(df_trng[features_trng])

    X_trng = df_trng[features_estmn].values.reshape(-1, len(features_trng))

    y_trng= model_regularisation.predict(X_trng)

    y_trng =[max(0,x) for x in y_trng]


    prediction_trained_model['prediction_label']=y_trng





estimation_trained_model=estimation_trained_model.assign(Date=identf_cols_to_be_added['Date'],\
                                                         site=identf_cols_to_be_added['site'])

estimation_trained_model=estimation_trained_model[['total_trip_count','Date','prediction_label','site']]



prediction_trained_model=prediction_trained_model.assign(Date=identf_cols_to_be_added_train['Date'],\
                                                         site=identf_cols_to_be_added_train['site'],\
                                                         people_counter_data=identf_cols_to_be_added_train[target])

prediction_trained_model=prediction_trained_model[['total_trip_count',target,'Date','prediction_label','site']]

In [ ]:
estimation_trained_model.to_pickle(data_folder+'estimation_trained_model.pkl')

prediction_trained_model.to_pickle(data_folder+'prediction_trained_model.pkl')

In [ ]:
# heat-map for a specific season
estimation_trained_model['season']=estimation_trained_model['Date'].apply(lambda x: x.split('-')[1]).\
apply(lambda x : get_season(x))
est_sesn=estimation_trained_model[estimation_trained_model['season']=='summer']

est_sesn=est_sesn.groupby('site')[['total_trip_count','prediction_label']].mean().reset_index()

est_sesn=est_sesn.merge(counter_cr,left_on=['site'],right_on='counter',how='inner')

del est_sesn['counter']



# Estimated
fig = px.density_mapbox(est_sesn, lat='latitude', lon='longitude', z='prediction_label',
                        mapbox_style="stamen-terrain")
 
fig

In [ ]:
# Strava
fig = px.density_mapbox(est_sesn, lat='latitude', lon='longitude', z='total_trip_count',
                        mapbox_style="stamen-terrain")
 
fig

**Visualisations**

In [ ]:
# Average value across all sites

estimation_ntnl=estimation_trained_model.groupby('Date')[['total_trip_count','prediction_label']].mean()



prediction_trn_ntnl=prediction_trained_model.groupby('Date')[['total_trip_count',target,'prediction_label']].mean()

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(y=estimation_ntnl['prediction_label'], x=estimation_ntnl.index, name="Estimated data"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(y=estimation_ntnl['total_trip_count'], x=estimation_ntnl.index, name="Strava data"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Estimated visitor count for similar sites"
)

# Set x-axis title
fig.update_xaxes(title_text="Time")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Mean estimated count</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Mean Strava count</b>", secondary_y=True)

fig.show()

In [ ]:
analysis = estimation_ntnl[['prediction_label']].copy()


decompose_result_mult = seasonal_decompose(analysis, model="multiplicative",period=4)

trend = decompose_result_mult.trend
seasonal = decompose_result_mult.seasonal
residual = decompose_result_mult.resid

#decompose_result_mult.plot();

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(y=analysis['prediction_label'], x=analysis.index, name="Predicted data"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(y=trend.values,x=trend.index, name="Predicted trend"),
    secondary_y=True,
)


# Add figure title
fig.update_layout(
    title_text="Estimated visitor count for similar sites"
)

# Set x-axis title
fig.update_xaxes(title_text="Time")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Visitor count</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Visitor trend</b>", secondary_y=True)

fig.show()

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(y=prediction_trn_ntnl['prediction_label'], x=prediction_trn_ntnl.index, name="Predicted data"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(y=prediction_trn_ntnl[target],x=prediction_trn_ntnl.index, name="Actual data"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(y=prediction_trn_ntnl['total_trip_count'], x=prediction_trn_ntnl.index, name="Strava data"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Visitor count for sites in the training data"
)

# Set x-axis title
fig.update_xaxes(title_text="Time")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Mean count</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Mean Strava count</b>", secondary_y=True)

fig.show()

In [ ]:
analysis = prediction_trn_ntnl[['prediction_label']].copy()


decompose_result_mult = seasonal_decompose(analysis, model="additive",period=4)

trend = decompose_result_mult.trend
seasonal = decompose_result_mult.seasonal
residual = decompose_result_mult.resid

#decompose_result_mult.plot();

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(y=trend, x=analysis.index, name="Predicted trend"),
    secondary_y=False,
)


analysis = prediction_trn_ntnl[[target]].copy()


decompose_result_mult = seasonal_decompose(analysis, model="multiplicative",period=4)

trend = decompose_result_mult.trend
seasonal = decompose_result_mult.seasonal
residual = decompose_result_mult.resid

#decompose_result_mult.plot();

fig.add_trace(
    go.Scatter(y=trend.values,x=trend.index, name="Actual trend"),
    secondary_y=True,
)


# Add figure title
fig.update_layout(
    title_text="Visitor count for sites in the training data"
)

# Set x-axis title
fig.update_xaxes(title_text="Time")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Predicted trend</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Actual trend</b>", secondary_y=True)

fig.show()

In [ ]:
all_sites_count=pd.read_pickle(data_folder+'complete_dataset.pkl')


all_sites_count=all_sites_count[all_sites_count['site'].isin(places_ne+places_nd+places_cr)]

all_sites_count_avg=all_sites_count.groupby('Date')['total_trip_count'].sum()

analysis = all_sites_count_avg.values


decompose_result_mult = seasonal_decompose(analysis, model="multiplicative",period=4)

trend = decompose_result_mult.trend
seasonal = decompose_result_mult.seasonal
residual = decompose_result_mult.resid

#decompose_result_mult.plot();

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(y=trend, x=all_sites_count_avg.index, name="Trend"),
    secondary_y=False,
)


analysis = all_sites_count_avg.copy()



#decompose_result_mult.plot();

fig.add_trace(
    go.Scatter(y=all_sites_count_avg.values,x=all_sites_count_avg.index, name="Observed count"),
    secondary_y=True,
)


# Add figure title
fig.update_layout(
    title_text="Mean Strava count for all sites"
)

# Set x-axis title
fig.update_xaxes(title_text="Time")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Observed trend</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Observed data</b>", secondary_y=True)

fig.show()

In [ ]:
all_sites_count_avg=all_sites_count.groupby('Date')[target].mean()

analysis = all_sites_count_avg.values


decompose_result_mult = seasonal_decompose(analysis, model="multiplicative",period=6)

trend = decompose_result_mult.trend
seasonal = decompose_result_mult.seasonal
residual = decompose_result_mult.resid

#decompose_result_mult.plot();

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(y=trend, x=all_sites_count_avg.index, name="Trend"),
    secondary_y=False,
)


analysis = all_sites_count_avg.copy()



#decompose_result_mult.plot();

fig.add_trace(
    go.Scatter(y=all_sites_count_avg.values,x=all_sites_count_avg.index, name="Observed count"),
    secondary_y=True,
)


# Add figure title
fig.update_layout(
    title_text="Mean visitor count for all sites"
)

# Set x-axis title
fig.update_xaxes(title_text="Time")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Observed trend</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Observed data</b>", secondary_y=True)

fig.show()

In [ ]:
all_sites_count=pd.read_pickle(nature_srvy_visits)

all_sites_count.rename(columns={'counter':'site'},inplace=True)

all_sites_count=all_sites_count[all_sites_count['site'].isin(places_ne+places_nd+places_cr)]

all_sites_count_avg=all_sites_count.groupby('Date')['No_Of_Visits'].mean()

analysis = all_sites_count_avg


decompose_result_mult = seasonal_decompose(analysis, model="multiplicative",period=4)

trend = decompose_result_mult.trend
seasonal = decompose_result_mult.seasonal
residual = decompose_result_mult.resid

all_sites_count_avg.index=all_sites_count_avg.index.astype(str)

#decompose_result_mult.plot();

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(y=trend, x=all_sites_count_avg.index, name="Trend"),
    secondary_y=False,
)


analysis = all_sites_count_avg.copy()



#decompose_result_mult.plot();

fig.add_trace(
    go.Scatter(y=all_sites_count_avg.values,x=all_sites_count_avg.index, name="Observed count"),
    secondary_y=True,
)


# Add figure title
fig.update_layout(
    title_text="Mean People and Nature Survey count for all sites"
)

# Set x-axis title
fig.update_xaxes(title_text="Time")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Observed trend</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Observed data</b>", secondary_y=True)

fig.show()